 GroupBy机制

Hadley Wickham（许多热门R# 第10章 数据聚合与分组运算

对数据集进行分组并对各组应用一个函数（无论是聚合还是转换），通常是数据分析工作中的重要环节。在将数据集加载、融合、准备好之后，通常就是计算分组统计或生成透视表。pandas提供了一个灵活高效的gruopby功能，它使你能以一种自然的方式对数据集进行切片、切块、摘要等操作。

关系型数据库和SQL（Structured Query Language，结构化查询语言）能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合。但是，像SQL这样的查询语言所能执行的分组运算的种类很有限。在本章中你将会看到，由于Python和pandas强大的表达能力，我们可以执行复杂得多的分组运算（利用任何可以接受pandas对象或NumPy数组的函数）。在本章中，你将会学到：

- 使用一个或多个键（形式可以是函数、数组或DataFrame列名）分割pandas对象。
- 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
- 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
- 计算透视表或交叉表。
- 执行分位数分析以及其它统计分组分析。

>笔记：对时间序列数据的聚合（groupby的特殊用法之一）也称作重采样（resampling），本书将在第11章中单独对其进行讲解。

# 10.1 GroupBy机制

Hadley Wickham（许多热门R语言包的作者）创造了一个用于表示分组运算的术语"split-apply-combine"（拆分－应用－合并）。第一个阶段，pandas对象（无论是Series、DataFrame还是其他的）中的数据会根据你所提供的一个或多个键被拆分（split）为多组。拆分操作是在对象的特定轴上执行的。例如，DataFrame可以在其行（axis=0）或列（axis=1）上进行分组。然后，将一个函数应用（apply）到各个分组并产生一个新值。最后，所有这些函数的执行结果会被合并（combine）到最终的结果对象中。结果对象的形式一般取决于数据上所执行的操作。图10-1大致说明了一个简单的分组聚合过程。语言包的作者）创造了一个用于表示分组运算的术语"split-apply-combine"（拆分－应用－合并）。第一个阶段，pandas对象（无论是Series、DataFrame还是其他的）中的数据会根据你所提供的一个或多个键被拆分（split）为多组。拆分操作是在对象的特定轴上执行的。例如，DataFrame可以在其行（axis=0）或列（axis=1）上进行分组。然后，将一个函数应用（apply）到各个分组并产生一个新值。最后，所有这些函数的执行结果会被合并（combine）到最终的结果对象中。结果对象的形式一般取决于数据上所执行的操作。图10-1大致说明了一个简单的分组聚合过程。

分组键可以有多种形式，且类型不必相同：

- 列表或数组，其长度与待分组的轴一样。
- 表示DataFrame某个列名的值。
- 字典或Series，给出待分组轴上的值与分组名之间的对应关系。
- 函数，用于处理轴索引或索引中的各个标签。

注意，后三种都只是快捷方式而已，其最终目的仍然是产生一组用于拆分对象的值。如果觉得这些东西看起来很抽象，不用担心，我将在本章中给出大量有关于此的示例。首先来看看下面这个非常简单的表格型数据集（以DataFrame的形式）：

In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-0.709572,-2.244020
1,a,two,-0.207853,-0.695912
2,b,one,0.603908,-1.278808
3,b,two,-1.152423,0.973140
4,a,one,-1.283533,1.729247


假设你想要按key1进行分组，并计算data1列的平均值。实现该功能的方式有很多，而我们这里要用的是：访问data1，并根据key1调用groupby：

In [4]:
grouped = df['data1'].groupby(df['key1'])

grouped

变量grouped是一个GroupBy对象。它实际上还没有进行任何计算，只是含有一些有关分组键df['key1']的中间数据而已。换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。例如，我们可以调用GroupBy的mean方法来计算分组平均值：

In [5]:
# 分组参数打印
grouped.mean()

key1
a    0.761634
b    1.017565
Name: data1, dtype: float64

稍后我将详细讲解.mean()的调用过程。这里最重要的是，数据（Series）根据分组键进行了聚合，产生了一个新的Series，其索引为key1列中的唯一值。之所以结果中索引的名称为key1，是因为原始DataFrame的列df['key1']就叫这个名字。

如果我们一次传入多个数组的列表，就会得到不同的结果：

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

means

key1  key2
a     one     0.772069
      two     0.740763
b     one    -0.063525
      two     2.098655
Name: data1, dtype: float64

这里，我通过两个键对数据进行了分组，得到的Series具有一个层次化索引（由唯一的键对组成）：

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.772069,0.740763
b,-0.063525,2.098655


在这个例子中，分组键均为Series。实际上，分组键可以是任何长度适当的数组：

In [11]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

print(states)
print(years)

['Ohio' 'California' 'California' 'Ohio' 'Ohio']
[2005 2005 2006 2005 2006]


In [12]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.207853
            2006    0.603908
Ohio        2005   -0.930997
            2006   -1.283533
Name: data1, dtype: float64

通常，分组信息就位于相同的要处理DataFrame中。这里，你还可以将列名（可以是字符串、数字或其他Python对象）用作分组键：

In [13]:
df.groupby('key1').mean()

/tmp/ipykernel_85770/2057825055.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean()


,data1,data2
key1,,
a,-0.733653,-0.403562
b,-0.274257,-0.152834


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.996553 -0.257387
     two  -0.207853 -0.695912
b    one   0.603908 -1.278808
     two  -1.152423  0.973140

你可能已经注意到了，第一个例子在执行df.groupby('key1').mean()时，结果中没有key2列。这是因为df['key2']不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合，虽然有时可能会被过滤为一个子集，稍后就会碰到。

无论你准备拿groupby做什么，都有可能会用到GroupBy的size方法，它可以返回一个含有分组大小的Series：

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

注意，任何分组关键词中的缺失值，都会被从结果中除去。

## 对分组进行迭代

GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。看下面的例子：

In [19]:
for name, group in df.groupby('key1'):
    print("name: ", name)
    print("group: \n", group)
    print("")

name:  a
group: 
   key1 key2     data1     data2
0    a  one -0.709572 -2.244020
1    a  two -0.207853 -0.695912
4    a  one -1.283533  1.729247

name:  b
group: 
   key1 key2     data1     data2
2    b  one  0.603908 -1.278808
3    b  two -1.152423  0.973140



对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

In [25]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print('key: ', (k1, k2))
    print("group: \n", group)
    print("")

key:  ('a', 'one')
group: 
   key1 key2     data1     data2
0    a  one -0.709572 -2.244020
4    a  one -1.283533  1.729247

key:  ('a', 'two')
group: 
   key1 key2     data1     data2
1    a  two -0.207853 -0.695912

key:  ('b', 'one')
group: 
   key1 key2     data1     data2
2    b  one  0.603908 -1.278808

key:  ('b', 'two')
group: 
   key1 key2     data1    data2
3    b  two -1.152423  0.97314



当然，你可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：将这些数据片段做成一个字典：

In [26]:
pieces = dict(list(df.groupby('key1')))

pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.709572 -2.244020
 1    a  two -0.207853 -0.695912
 4    a  one -1.283533  1.729247,
 'b':   key1 key2     data1     data2
 2    b  one  0.603908 -1.278808
 3    b  two -1.152423  0.973140}

In [27]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.603908,-1.278808
3,b,two,-1.152423,0.973140


groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。拿上面例子中的df来说，我们可以根据dtype对列进行分组：

In [28]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

可以如下打印分组：

In [31]:
for dtype, group in grouped:
    print("dtype: ", dtype)
    print("group: ", group)
    print("")

dtype:  a
group:  0    0.310221
1    0.740763
4    1.233916
Name: data1, dtype: float64

dtype:  b
group:  2   -0.063525
3    2.098655
Name: data1, dtype: float64



## 选取一列或列的子集

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。也就是说：

In [32]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

是以下代码的语法糖：

In [33]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写：

In [34]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.257387
     two  -0.695912
b    one  -1.278808
     two   0.973140

这种索引操作所返回的对象是一个已分组的DataFrame（如果传入的是列表或数组）或已分组的Series（如果传入的是标量形式的单个列名）：

In [36]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [37]:
s_grouped.mean()

key1  key2
a     one    -0.257387
      two    -0.695912
b     one    -1.278808
      two     0.973140
Name: data2, dtype: float64

In [41]:
for name, value in s_grouped:
    print("name: ", name)
    print("value: \n", value)
    print("")

name:  ('a', 'one')
value: 
 0   -2.244020
4    1.729247
Name: data2, dtype: float64

name:  ('a', 'two')
value: 
 1   -0.695912
Name: data2, dtype: float64

name:  ('b', 'one')
value: 
 2   -1.278808
Name: data2, dtype: float64

name:  ('b', 'two')
value: 
 3    0.97314
Name: data2, dtype: float64



## 通过字典或Series进行分组

除数组以外，分组信息还可以其他形式存在。来看另一个示例DataFrame：

In [42]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people

,a,b,c,d,e
Joe,1.454533,0.014873,-0.122356,-1.347580,0.259870
Steve,2.569880,1.112693,1.627678,0.344245,-2.119006
Wes,-0.607567,0.248615,-0.777997,0.803140,-0.999852
Jim,-0.516609,-1.164168,2.039728,0.006648,-0.336257
Travis,0.200200,1.318189,1.247452,0.601178,-1.009509


In [43]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values

In [44]:
people

,a,b,c,d,e
Joe,1.454533,0.014873,-0.122356,-1.347580,0.259870
Steve,2.569880,1.112693,1.627678,0.344245,-2.119006
Wes,-0.607567,NaN,NaN,0.803140,-0.999852
Jim,-0.516609,-1.164168,2.039728,0.006648,-0.336257
Travis,0.200200,1.318189,1.247452,0.601178,-1.009509


现在，假设已知列的分组关系，并希望根据分组计算列的和：

In [45]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

现在，你可以将这个字典传给groupby，来构造数组，但我们可以直接传递字典（我包含了键“f”来强调，存在未使用的分组键是可以的）：

In [47]:
by_column = people.groupby(mapping, axis=1)

by_column.sum()

,blue,red
Joe,-1.469937,1.729276
Steve,1.971923,1.563566
Wes,0.803140,-1.607419
Jim,2.046376,-2.017034
Travis,1.848630,0.508879


Series也有同样的功能，它可以被看做一个固定大小的映射：

In [48]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [49]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 通过函数进行分组

比起使用字典或Series，使用Python函数是一种更原生的方法定义分组映射。任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。具体点说，以上一小节的示例DataFrame为例，其索引值为人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入len函数：

In [52]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])


people

,a,b,c,d,e
Joe,-1.515187,-0.293011,0.043250,-0.993962,-1.422512
Steve,-1.237449,-1.324410,-0.346604,1.594054,1.449758
Wes,-1.144937,0.933168,-0.473135,-1.252821,-1.721972
Jim,0.293474,-0.554137,-0.469367,-0.311950,-0.443493
Travis,0.987851,-0.379189,-0.269205,-1.351622,-0.338383


In [ ]:
people.groupby(len).sum()

将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组：

In [54]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.515187 -0.293011 -0.473135 -1.252821 -1.721972
  two  0.293474 -0.554137 -0.469367 -0.311950 -0.443493
5 one -1.237449 -1.324410 -0.346604  1.594054  1.449758
6 two  0.987851 -0.379189 -0.269205 -1.351622 -0.338383

## 根据索引级别分组

层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

In [55]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [58]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)


hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.021959  0.105497  0.022978  0.820139  1.716738
1      0.157147 -1.131472 -0.513811  2.601009  0.009398
2      0.646493 -0.110365 -1.499775 -0.464107  0.703853
3      0.484224 -1.145448  0.083226  0.238550 -0.707838

要根据级别分组，使用level关键字传递级别序号或名字：

In [59]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 数据聚合

聚合指的是任何能够从数组产生标量值的数据转换过程。之前的例子已经用过一些，比如mean、count、min以及sum等。你可能想知道在GroupBy对象上调用mean()时究竟发生了什么。许多常见的聚合运算（如表10-1所示）都有进行优化。然而，除了这些方法，你还可以使用其它的。

你可以使用自己发明的聚合运算，还可以调用分组对象上已经定义好的任何方法。例如，quantile可以计算Series或DataFrame列的样本分位数。

虽然quantile并没有明确地实现于GroupBy，但它是一个Series方法，所以这里是能用的。实际上，GroupBy会高效地对Series进行切片，然后对各片调用piece.quantile(0.9)，最后将这些结果组装成最终结果：

In [1]:
df

NameError: name 'df' is not defined

In [63]:
grouped = df.groupby('key1')

grouped['data1'].quantile(0.9)  # 分位数

key1
a   -0.308197
b    0.428275
Name: data1, dtype: float64

如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可：

In [65]:
def peak_to_peak(arr):
    return arr.max() - arr.min()


grouped.agg(peak_to_peak)

/tmp/ipykernel_85770/1559344061.py:5: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,1.075680,3.973267
b,1.756331,2.251948


你可能注意到注意，有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：

In [66]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.733653  0.538244 -1.283533 -0.996553 -0.709572 -0.458713   
b      2.0 -0.274257  1.241914 -1.152423 -0.713340 -0.274257  0.164826   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.207853   3.0 -0.403562  2.002701 -2.244020 -1.469966 -0.695912   
b     0.603908   2.0 -0.152834  1.592367 -1.278808 -0.715821 -0.152834   

                          
           75%       max  
key1                      
a     0.516667  1.729247  
b     0.410153  0.973140

在后面的10.3节，我将详细说明这到底是怎么回事。

>笔记：自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

## 面向列的多函数应用

回到前面小费的例子。使用read_csv导入数据之后，我们添加了一个小费百分比的列tip_pct：

In [71]:
tips = pd.read_csv('../../examples/tips.csv')


tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [72]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


你已经看到，对Series或DataFrame列的聚合运算其实就是使用aggregate（使用自定义函数）或调用诸如mean、std之类的方法。然而，你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数。其实这也好办，我将通过一些示例来进行讲解。首先，我根据天和smoker对tips进行分组：

In [73]:
grouped = tips.groupby(['day', 'smoker'])

注意，对于表10-1中的那些描述统计，可以将函数名以字符串的形式传入：

In [75]:
grouped_pct = grouped['tip_pct']

grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名：

In [76]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

这里，我们传递了一组聚合函数进行聚合，独立对数据分组进行评估。

你并非一定要接受GroupBy自动给出的那些列名，特别是lambda函数，它们的名称是'<lambda>'，这样的辨识度就很低了（通过函数的__name__属性看看就知道了）。因此，如果传入的是一个由(name,function)元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序映射）：

In [77]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

对于DataFrame，你还有更多选择，你可以定义一组应用于全部列的一组函数，或不同的列应用不同的函数。假设我们想要对tip_pct和total_bill列计算三个统计信息：

In [79]:
functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)

result

/tmp/ipykernel_85770/1864244588.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

如你所见，结果DataFrame拥有层次化的列，这相当于分别对各列进行聚合，然后用concat将结果组装到一起，使用列名用作keys参数：

In [80]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

跟前面一样，这里也可以传入带有自定义名称的一组元组：

In [82]:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

/tmp/ipykernel_85770/714931474.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

现在，假设你想要对一个列或不同的列应用不同的函数。具体的办法是向agg传入一个从列名映射到函数的字典：

In [83]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [84]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

只有将多个函数应用到至少一列时，DataFrame才会拥有层次化的列。

## 以“没有行索引”的形式返回聚合数据

到目前为止，所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）。由于并不总是需要如此，所以你可以向groupby传入as_index=False以禁用该功能：

In [85]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

/tmp/ipykernel_85770/2496195394.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(['day', 'smoker'], as_index=False).mean()


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


当然，对结果调用reset_index也能得到这种形式的结果。使用as_index=False方法可以避免一些不必要的计算。